In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

In [19]:
# path = '/content/drive/MyDrive/Colab Notebooks/dataset'
path = 'C:/dataset'

In [20]:
# import zipfile
# zip_file = zipfile.ZipFile(path+'/osteoarthritis.zip') # 압축을 해제할 '/파일경로/파일명.zip'
# zip_file.extractall('/content/drive/MyDrive/Colab Notebooks/dataset/')

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import math
import cv2

In [22]:
categories = ['train', 'test', 'val', 'auto_test'] # 전처리된 데이터셋을 훈련용, 평가용, 검증용으로 구분
data_dir = path+'/osteoarthritis/'
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu') # Mac OS
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )

In [23]:
def resize_image(img,size=(128,128)):
    return cv2.resize(img,size)

def he_img(img):
    return cv2.equalizeHist(img)

def clahe_image(img):
    clahe = cv2.createCLAHE(clipLimit=2.,tileGridSize=(8,8))
    cl_img = clahe.apply(img)
    return cl_img

def denoise_img(img):
    return cv2.fastNlMeansDenoising(img,None,30,7,21)

def normalize_img(img):
    return cv2.normalize(img,None,0,255,cv2.NORM_MINMAX)

def detect_edge(img):
    return cv2.Canny(img,100,200)

def blur_img(img):
    return cv2.GaussianBlur(img,(5,5),0)

def find_contour(img):
    ret, thresh = cv2.threshold(img, 127, 255, 0)
    contours, hiearchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [24]:
# 데이터 증강

In [25]:
def load_data(data_dir):
    images = []
    for img_name in os.listdir(data_dir):
        img_path = os.path.join(data_dir, img_name)
        if os.path.isfile(img_path):  # 파일인지 확인
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:  # 이미지가 정상적으로 로드되었는지 확인
                img = resize_image(img)
                img = clahe_image(img)
                img = normalize_img(img)
                images.append(img)
    prepared_data = np.array(images)
    return prepared_data

# 증강 or 클래스 별 동일한 숫자
# 전체적으로 분류할 때 비율을 맞춰보는게..
# 증강(rotation, zoomin)


In [26]:
all_data = {}

# 각 카테고리와 라벨에 따라 이미지를 처리
for category in categories:
    category_path = os.path.join(data_dir, category)
    all_data[category] = {}
    for label in range(5):
        label_path = os.path.join(category_path, str(label))
        if os.path.isdir(label_path):  # 라벨 경로가 디렉토리인지 확인
            print(f"카테고리: {category}, 라벨: {label} 처리 중....")
            processed_img_list = load_data(label_path)
            all_data[category][label] = processed_img_list
            print(f"처리된 이미지 수: {len(processed_img_list)}")
            print('='*40)

카테고리: train, 라벨: 0 처리 중....
처리된 이미지 수: 2286
카테고리: train, 라벨: 1 처리 중....
처리된 이미지 수: 1046
카테고리: train, 라벨: 2 처리 중....
처리된 이미지 수: 1516
카테고리: train, 라벨: 3 처리 중....
처리된 이미지 수: 757
카테고리: train, 라벨: 4 처리 중....
처리된 이미지 수: 173
카테고리: test, 라벨: 0 처리 중....
처리된 이미지 수: 639
카테고리: test, 라벨: 1 처리 중....
처리된 이미지 수: 296
카테고리: test, 라벨: 2 처리 중....
처리된 이미지 수: 447
카테고리: test, 라벨: 3 처리 중....
처리된 이미지 수: 223
카테고리: test, 라벨: 4 처리 중....
처리된 이미지 수: 51
카테고리: val, 라벨: 0 처리 중....
처리된 이미지 수: 328
카테고리: val, 라벨: 1 처리 중....
처리된 이미지 수: 153
카테고리: val, 라벨: 2 처리 중....
처리된 이미지 수: 212
카테고리: val, 라벨: 3 처리 중....
처리된 이미지 수: 106
카테고리: val, 라벨: 4 처리 중....
처리된 이미지 수: 27
카테고리: auto_test, 라벨: 0 처리 중....
처리된 이미지 수: 604
카테고리: auto_test, 라벨: 1 처리 중....
처리된 이미지 수: 275
카테고리: auto_test, 라벨: 2 처리 중....
처리된 이미지 수: 403
카테고리: auto_test, 라벨: 3 처리 중....
처리된 이미지 수: 200
카테고리: auto_test, 라벨: 4 처리 중....
처리된 이미지 수: 44


In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset


# train, val, test 데이터를 모두 합침
combined_data = []
combined_labels = []

for dataset in ['train', 'val', 'test']:
    for label, images in all_data[dataset].items():
        combined_data.append(images)
        combined_labels.append(np.full(images.shape[0], label))

combined_data = np.concatenate(combined_data, axis=0)
combined_labels = np.concatenate(combined_labels, axis=0)

# 80/10/10 비율로 train, val, test 세트를 나눔
train_data, test_data, train_labels, test_labels = train_test_split(combined_data, combined_labels, test_size=0.2, random_state=42)
val_data, test_data, val_labels, test_labels = train_test_split(test_data, test_labels, test_size=0.5, random_state=42)

# PyTorch 텐서로 변환
train_data_tensor = torch.tensor(train_data, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
val_data_tensor = torch.tensor(val_data, dtype=torch.float32)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)
test_data_tensor = torch.tensor(test_data, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# PyTorch 데이터셋 및 데이터 로더 생성
train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_data_tensor, val_labels_tensor)
test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [28]:
train_data.shape, test_data.shape, train_labels.shape, test_labels.shape

((6608, 128, 128), (826, 128, 128), (6608,), (826,))

In [29]:
val_data.shape, test_data.shape, val_labels.shape, test_labels.shape

((826, 128, 128), (826, 128, 128), (826,), (826,))

In [30]:
# Define the number of output classes
num_classes = 5

# Load pre-trained models and modify the final layer for transfer learning
def get_pretrained_model(model_name, num_classes):
    if model_name == 'resnet':
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == 'densenet':
        model = models.densenet121(pretrained=True)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    elif model_name == 'vgg':
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    else:
        raise ValueError('Unknown model name')
    
    return model

# 사전학습 모델 설정
criterion = nn.CrossEntropyLoss()
model_name = 'resnet'  # or 'densenet' or 'vgg'
model = get_pretrained_model(model_name, num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Freeze initial layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last layer
if model_name == 'resnet':
    for param in model.fc.parameters():
        param.requires_grad = True
elif model_name == 'densenet':
    for param in model.classifier.parameters():
        param.requires_grad = True
elif model_name == 'vgg':
    for param in model.classifier[6].parameters():
        param.requires_grad = True

# Redefine optimizer to update only the last layer
optimizer = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()), lr=0.001)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Ensure inputs have the correct shape (N, C, H, W) and convert grayscale to RGB
        if inputs.ndim == 3:
            inputs = inputs.unsqueeze(1)  # Add channel dimension if missing
        if inputs.shape[1] == 1:
            inputs = inputs.repeat(1, 3, 1, 1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            
            # Ensure images have the correct shape (N, C, H, W) and convert grayscale to RGB
            if images.ndim == 3:
                images = images.unsqueeze(1)  # Add channel dimension if missing
            if images.shape[1] == 1:
                images = images.repeat(1, 3, 1, 1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch + 1}, Validation loss: {val_loss / len(val_loader):.3f}, Accuracy: {100 * correct / total:.2f}%')

print('Training complete')

# Testing phase
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        # Ensure images have the correct shape (N, C, H, W) and convert grayscale to RGB
        if images.ndim == 3:
            images = images.unsqueeze(1)  # Add channel dimension if missing
        if images.shape[1] == 1:
            images = images.repeat(1, 3, 1, 1)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')



RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 32, 128, 128] to have 3 channels, but got 32 channels instead

In [ ]:
# # 테스트 단계
# model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in test_loader:
#         images, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f'Test Accuracy: {100 * correct / total:.2f}%')